##MNIST數據預處理

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 超參數
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# MNIST數據預處理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)



## 定義ANN模型

In [ ]:

# 定義ANN模型
class ANN(nn.Module):
    def __init__(self, dropout_rate):
        super(ANN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = ANN()

## 訓練和評估，同時繪製Loss和Accuracy變化趨勢

In [ ]:

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 訓練和評估，同時繪製Loss和Accuracy變化趨勢
loss_history = []
accuracy_history = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    loss_history.append(total_loss / len(train_loader))
    accuracy_history.append(accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}] - Loss: {loss_history[-1]:.4f}, Accuracy: {accuracy:.4f}')


## 繪製Loss和Accuracy變化趨勢

In [ ]:

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(loss_history)
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Value')

plt.subplot(1, 2, 2)
plt.plot(accuracy_history)
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')

plt.tight_layout()
plt.show()